<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center><font color= #004A94> <font size = 6> Ingeniería Financiera </font> <br>

<center> <font color= #004A94> <font size = 5> Microestructura y Sistemas de Trading </font> <br> <br> <font color= #047CFB> <font size = 5> Jose Luis Suárez de Alba </font> <br> <br> <font color= #047CFB> <font size = 5> Axel Francisco Munguía Quintero</font>  <br> <br> <font color= #047CFB> <font size = 5> Hannia Celilia Lomeli Salado </font> <br> <br> <font color= #047CFB> <font size = 5> Paulina Milenka Glez Manrique </font> 

<center><font color= #42c29b> <font size = 6> Proyecto Final </font> <br> <br> <font color= #047CFB> <font size = 5>Repositorio: https://github.com/hannylomeli/FinalTrading_EquipoA <a href='https://github.com/'>Link</a></font>

## Obejtivo


Diseñar una estrategia de trading con base a la reacción del precio ante el comunicado de indicadores macroeconómicos (Análisis fundamental). Lo anterior, utilizando lo que se conoce como "Calendario Económico", y más en particular, aquel que incorpora información de las principales economías del mundo por lo que es ampliamiente utilizado en el mercado de Forex.

#### Importando datos

In [74]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
import g_boxplot_varios as g

#### Calendario económico

In [75]:
data = pd.read_csv('calendario_economico.csv')
data['timestamp'] = [pd.to_datetime(data.iat[i,0]).tz_localize('GMT') for i in range(len(data))] # Timestamp a 'GMT'
data.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus
0,2009-01-05 09:30:00+00:00,EUR,Sentix Investor Confidence,European Monetary Union,1,-34.4,-42.3,NaN
1,2009-01-05 15:00:00+00:00,USD,Construction Spending (MoM),United States,1,-1.2,-0.4,-1.3
2,2009-01-06 08:55:00+00:00,EUR,Markit Services PMI,Germany,1,46.6,46.4,46.4
3,2009-01-06 09:00:00+00:00,EUR,Markit Services PMI,European Monetary Union,0,42.1,42.0,42.0
4,2009-01-06 10:00:00+00:00,EUR,Consumer Price Index (YoY),European Monetary Union,2,1.6,2.1,1.8


#### Precios eur/usd

In [76]:
p = pd.read_csv('precios_historicos_eurusd.csv')
p['timestamp'] = [pd.to_datetime(p.iat[i,0]).tz_localize('GMT') for i in range(len(p))] # Timestamp a 'GMT'
p.head()

,timestamp,open,high,low,close
0,2009-01-06 05:00:00+00:00,1.35564,1.35565,1.35383,1.35439
1,2009-01-06 05:05:00+00:00,1.35443,1.35489,1.35424,1.35474
2,2009-01-06 05:10:00+00:00,1.35480,1.35552,1.35439,1.35539
3,2009-01-06 05:15:00+00:00,1.35545,1.35549,1.35101,1.35101
4,2009-01-06 05:20:00+00:00,1.35101,1.35130,1.35021,1.35086


## Selección de indicadores

### DataFrame para indicador de USD 'Factory Orders (MoM)'

##### Descripción del indicador 
> Emitidos por la Oficina de Censo de EE.UU
Los pedidos de fabrica son una medida del total de pedidos de bienes duraderos y no duraderos, como envíos (ventas), inventarios y pedidos a nivel de fabricación que pueden ofrecer información sobre la inflación y el crecimiento en el sector manufacturero.

##### Una lectura alta se ve como positiva (o alcista) para el USD, mientras que una lectura baja se ve como negativa (o bajista).

In [77]:
# Creando dataframe con indicadores deseados
df_usa = data[(data['Country'] == "United States") & (data['Name'] == "Factory Orders (MoM)")].reset_index() # Filtrando DataFrame
df_usa = df_usa.drop(['index'], 1) # Eliminando columna de index viejo

In [78]:
df_usa.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus
0,2009-01-06 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-6.5,-6.0,-2.3
1,2009-02-05 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-4.9,-6.5,-3.0
2,2009-03-05 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-3.5,-4.9,-2.0
3,2009-04-02 14:00:00+00:00,USD,Factory Orders (MoM),United States,2,0.7,-3.5,-1.3
4,2009-05-01 14:00:00+00:00,USD,Factory Orders (MoM),United States,2,-0.9,0.7,-0.7


In [79]:
idx = df_usa[df_usa['consensus'].isna()].index # Encontrando row de los NaN en 'Consensus'
df_usa.iloc[idx,7] = df_usa.iloc[idx,6] # Reemplazando 'Consensus' con 'Previous'

#### Clasificación de las observaciones de los comunicados según el siguiente criterio:

    o A  Actual >= Consensus >= Previous
    o B  Actual >= Consensus < Previous
    o C  Actual < Consensus >= Previous
    o D  Actual < Consensus < Previous

In [80]:
df_usa['escenario'] = ['A' if (df_usa['actual'][i] >= df_usa['consensus'][i]) & (df_usa['consensus'][i] >= df_usa['previous'][i]) 
                       else 'B' if (df_usa['actual'][i] >= df_usa['consensus'][i]) & (df_usa['consensus'][i] < df_usa['previous'][i]) 
                       else 'C' if (df_usa['actual'][i] < df_usa['consensus'][i]) & (df_usa['consensus'][i] >= df_usa['previous'][i])
                       else 'D' if (df_usa['actual'][i] < df_usa['consensus'][i]) & (df_usa['consensus'][i] < df_usa['previous'][i])
                       else 'None' for i in range(len(df_usa))] # Escenarios del calendario economico
df_usa['direccion'], df_usa['pips alcistas'], df_usa['pips bajistas'], df_usa['volatilidad'] = ["","","",""] # Creando columnas de metricos
df_usa.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,pips alcistas,pips bajistas,volatilidad
0,2009-01-06 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-6.5,-6.0,-2.3,C,,,,
1,2009-02-05 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-4.9,-6.5,-3.0,C,,,,
2,2009-03-05 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-3.5,-4.9,-2.0,C,,,,
3,2009-04-02 14:00:00+00:00,USD,Factory Orders (MoM),United States,2,0.7,-3.5,-1.3,A,,,,
4,2009-05-01 14:00:00+00:00,USD,Factory Orders (MoM),United States,2,-0.9,0.7,-0.7,D,,,,


### DataFrame para indicador de EUR 'Industrial Production s.a. (MoM)'

##### Descripción del indicador 

> La producción industrial que publica la oficina de estadísticas de la Comisión Europea, Eurostat, mide la producción de las fábricas y minas de la Eurozona. Los cambios en la tendencia de la producción industrial se siguen con mucha atención como un indicador de la fortaleza del sector manufacturero.
#### Una lectura superior a lo anticipado es alcista para el euro, mientras que una lectura inferior es bajista.

In [81]:
# Creando dataframe con indicadores deseados
df_eur = data[(data['Country'] == "European Monetary Union") & (data['Name'] == "Industrial Production s.a. (MoM)")].reset_index() # Filtrando DataFrame
df_eur = df_eur.drop(['index'], 1) # Eliminando columna de index viejo

#### Clasificación de las observaciones de los comunicados según el siguiente criterio:

    o A  Actual >= Consensus >= Previous
    o B  Actual >= Consensus < Previous
    o C  Actual < Consensus >= Previous
    o D  Actual < Consensus < Previous

In [82]:
df_eur['escenario'] = ['A' if (df_eur['actual'][i] >= df_eur['consensus'][i]) & (df_eur['consensus'][i] >= df_eur['previous'][i]) 
                       else 'B' if (df_eur['actual'][i] >= df_eur['consensus'][i]) & (df_eur['consensus'][i] < df_eur['previous'][i]) 
                       else 'C' if (df_eur['actual'][i] < df_eur['consensus'][i]) & (df_eur['consensus'][i] >= df_eur['previous'][i])
                       else 'D' if (df_eur['actual'][i] < df_eur['consensus'][i]) & (df_eur['consensus'][i] < df_eur['previous'][i])
                       else 'None' for i in range(len(df_usa))] # Escenarios del calendario economico
df_eur['direccion'], df_eur['pips alcistas'], df_eur['pips bajistas'], df_eur['volatilidad'] = ["","","",""] # Creando columnas de metricos
df_eur.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,pips alcistas,pips bajistas,volatilidad
0,2009-01-14 10:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-3.0,-1.2,-1.8,D,,,,
1,2009-02-12 10:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-4.5,-3.0,-2.5,C,,,,
2,2009-03-20 10:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-2.9,-4.5,-1.5,C,,,,
3,2009-04-16 09:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-3.3,-2.9,-2.5,C,,,,
4,2009-05-13 09:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-1.1,-3.3,-1.0,C,,,,


### DataFrames por indicador

In [83]:
dates_usa = list(p.loc[p['timestamp'].isin(df_usa['timestamp'])]['timestamp']) # Timestamps presentes en indicador y precios

In [84]:
dates_usa[:10]

[Timestamp('2009-01-06 15:00:00+0000', tz='GMT'),
 Timestamp('2009-02-05 15:00:00+0000', tz='GMT'),
 Timestamp('2009-03-05 15:00:00+0000', tz='GMT'),
 Timestamp('2009-04-02 14:00:00+0000', tz='GMT'),
 Timestamp('2009-05-01 14:00:00+0000', tz='GMT'),
 Timestamp('2009-06-03 14:00:00+0000', tz='GMT'),
 Timestamp('2009-07-02 14:00:00+0000', tz='GMT'),
 Timestamp('2009-08-05 14:00:00+0000', tz='GMT'),
 Timestamp('2009-09-02 14:00:00+0000', tz='GMT'),
 Timestamp('2009-10-02 14:00:00+0000', tz='GMT')]

In [85]:
p.loc[p['timestamp'] == dates_usa[0]].index.values[0]

120

In [86]:
p_usa_df = {} # Diccionario para df de precios indicador usa
for i in range(len(dates_usa)):
    ref = p.loc[p['timestamp'] == dates_usa[i]].index.values[0] # Fila de donde se encuentra el timestamp del calendario en precios
    start = ref - 6 # Sacando 30 min antes (fila)
    end = ref + 6 # Sacando 30 min despues (fila)
    if start<0:
        start = 0
    elif end>p.shape[0]:
        enf = p.shape[0]
    df = p.iloc[start:end+1, :].reset_index().drop('index',1) # Filtrando dataframe, reinicio de index y drop del viejo index
    p_usa_df[dates_usa[i]] = df # Almacenar en diccionario con siendo el timestamp usado como 'key'

In [87]:
dates_eur = list(p.loc[p['timestamp'].isin(df_eur['timestamp'])]['timestamp']) # Timestamps presentes en indicador y precios

In [88]:
p_eur_df = {} # Diccionario para df de precios indicador eur
for i in range(len(dates_eur)):
    ref = p.loc[p['timestamp'] == dates_eur[i]].index.values[0] # Fila de donde se encuentra el timestamp del calendario en precios
    start = ref - 6 # Sacando 30 min antes (fila)
    end = ref + 6 # Sacando 30 min despues (fila)
    df = p.iloc[start:end+1, :].reset_index().drop('index',1) # Filtrando dataframe, reinicio de index y drop del viejo index
    p_eur_df[dates_eur[i]] = df # Almacenar en diccionario con siendo el timestamp usado como 'key'

### Métricas

In [89]:
def metricas(df_calendario, diccionario):
    for i in range(len(df_calendario)):
        df_calendario.loc[i,'direccion'] = diccionario[df_calendario['timestamp'][i]].loc[12,'close'] - diccionario[df_calendario['timestamp'][i]].loc[6,'open']
        df_calendario.loc[i,'pips alcistas'] = diccionario[df_calendario['timestamp'][i]].loc[6::,'high'].max() - diccionario[df_calendario['timestamp'][i]].loc[6,'open']
        df_calendario.loc[i,'pips bajistas'] = diccionario[df_calendario['timestamp'][i]].loc[6,'open'] - diccionario[df_calendario['timestamp'][i]].loc[6::,'low'].min()
        df_calendario.loc[i,'volatilidad'] = diccionario[df_calendario['timestamp'][i]]['high'].max() - diccionario[df_calendario['timestamp'][i]]['low'].min()
    return df_calendario

In [90]:
metricas_usa = metricas(df_usa, p_usa_df) # Metricas del indicador de USA
metricas_usa.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,pips alcistas,pips bajistas,volatilidad
0,2009-01-06 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-6.5,-6.0,-2.3,C,-0.00409,0.00284,0.00606,0.0089
1,2009-02-05 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-4.9,-6.5,-3.0,C,0.00177,0.00209,0.00109,0.00454
2,2009-03-05 15:00:00+00:00,USD,Factory Orders (MoM),United States,2,-3.5,-4.9,-2.0,C,0.00403,0.00411,0.00072,0.00706
3,2009-04-02 14:00:00+00:00,USD,Factory Orders (MoM),United States,2,0.7,-3.5,-1.3,A,-0.00304,0.00055,0.00423,0.00538
4,2009-05-01 14:00:00+00:00,USD,Factory Orders (MoM),United States,2,-0.9,0.7,-0.7,D,-0.00069,0.00102,0.0026,0.00383


In [91]:
fig = go.Figure()
fig = make_subplots()
fig.add_trace(go.Bar(x=list(metricas_usa['escenario'].value_counts().sort_index().index), 
                     y=metricas_usa['escenario'].value_counts().sort_index(),name='Conteo',
                         hovertemplate = '<i>Casos</i>: %{y}'
                        '<br><b>Identificador</b>: %{x}<br>'),secondary_y=False)
# Título del grafico
fig.update_layout(title_text='Escenarios USD Factory Orders (MoM)')
    
# Titulo eje x
fig.update_xaxes(title_text='Escenarios')
    
# Titulo eje y
fig.update_yaxes(title_text='<b>Conteo</b> de escenarios', secondary_y=False)
    
fig.show()

In [92]:
metricas_eur = metricas(df_eur, p_eur_df) # Metricas del indicador de EUR
metricas_eur.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,pips alcistas,pips bajistas,volatilidad
0,2009-01-14 10:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-3.0,-1.2,-1.8,D,0.00557,0.00557,0,0.00689
1,2009-02-12 10:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-4.5,-3.0,-2.5,C,-0.00484,0.00076,0.00544,0.00688
2,2009-03-20 10:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-2.9,-4.5,-1.5,C,-0.0069,0.0004,0.0069,0.00934
3,2009-04-16 09:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-3.3,-2.9,-2.5,C,-0.00341,0.00047,0.00359,0.00494
4,2009-05-13 09:00:00+00:00,EUR,Industrial Production s.a. (MoM),European Monetary Union,1,-1.1,-3.3,-1.0,C,0.00055,0.00099,0.0021,0.00383


In [93]:
fig = go.Figure()
fig = make_subplots()
fig.add_trace(go.Bar(x=list(metricas_eur['escenario'].value_counts().sort_index().index), 
                     y=metricas_eur['escenario'].value_counts().sort_index(),name='Conteo',
                         hovertemplate = '<i>Casos</i>: %{y}'
                        '<br><b>Identificador</b>: %{x}<br>'),secondary_y=False)
# Título del grafico
fig.update_layout(title_text='Escenarios EUR Industrial Production s.a. (MoM)')
    
# Titulo eje x
fig.update_xaxes(title_text='Escenarios')
    
# Titulo eje y
fig.update_yaxes(title_text='<b>Conteo</b> de escenarios', secondary_y=False)
    
fig.show()